In [17]:
import os
import joblib
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix

from app.core.utils import clean_text

In [18]:
DATA_PATH = "../dataset/email_text.csv"

df = pd.read_csv(DATA_PATH)
df.head()


,label,text
0,0,wrong bill grace i ' ll forward original messa...
1,0,i have continued the hilcorp old ocean deal da...
2,0,several related issues have resulted in an inc...
3,0,one year rate for this one will be escapenumbe...
4,0,attached is the weekly deal report for escapen...


In [19]:
df["clean_text"] = df["text"].apply(clean_text)

In [ ]:
X = df["clean_text"]
y = df["label"]

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.15, stratify=y, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.1765, stratify=y_temp, random_state=42
)

In [ ]:
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words="english",
        max_df=0.95,
        min_df=5,
        ngram_range=(1, 2),
        sublinear_tf=True
    )),
    ("clf", LinearSVC(class_weight="balanced"))
])

In [22]:
param_grid = {
    "tfidf__max_df": [0.9, 0.95],
    "tfidf__min_df": [3, 5],
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "clf__C": [0.5, 1, 2],
}

In [23]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv,
    scoring="f1",
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train, y_train)

best_model = grid.best_estimator_
print("Best Params:", grid.best_params_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END clf__C=0.5, tfidf__max_df=0.9, tfidf__min_df=3, tfidf__ngram_range=(1, 1); total time=   3.9s
[CV] END clf__C=0.5, tfidf__max_df=0.9, tfidf__min_df=3, tfidf__ngram_range=(1, 1); total time=   4.4s
[CV] END clf__C=0.5, tfidf__max_df=0.9, tfidf__min_df=3, tfidf__ngram_range=(1, 1); total time=   4.7s
[CV] END clf__C=0.5, tfidf__max_df=0.9, tfidf__min_df=3, tfidf__ngram_range=(1, 1); total time=   4.2s
[CV] END clf__C=0.5, tfidf__max_df=0.9, tfidf__min_df=3, tfidf__ngram_range=(1, 1); total time=   4.6s
[CV] END clf__C=0.5, tfidf__max_df=0.9, tfidf__min_df=5, tfidf__ngram_range=(1, 1); total time=   4.1s
[CV] END clf__C=0.5, tfidf__max_df=0.9, tfidf__min_df=5, tfidf__ngram_range=(1, 1); total time=   4.3s
[CV] END clf__C=0.5, tfidf__max_df=0.9, tfidf__min_df=5, tfidf__ngram_range=(1, 1); total time=   4.7s
[CV] END clf__C=0.5, tfidf__max_df=0.9, tfidf__min_df=5, tfidf__ngram_range=(1, 1); total time=   4.7s
[CV] END cl

In [24]:
val_preds = best_model.predict(X_val)
print(classification_report(y_val, val_preds))
print(confusion_matrix(y_val, val_preds))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99      2144
           1       0.98      1.00      0.99      2067

    accuracy                           0.99      4211
   macro avg       0.99      0.99      0.99      4211
weighted avg       0.99      0.99      0.99      4211

[[2109   35]
 [   8 2059]]


In [25]:
X_final = pd.concat([X_train, X_val])
y_final = pd.concat([y_train, y_val])

best_model.fit(X_final, y_final)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('tfidf', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"input input: {'filename', 'file', 'content'}, default='content'- If `'filename'`, the sequence passed as an argument to fit is expected to be a list of filenames that need reading to fetch the raw content to analyze.- If `'file'`, the sequence items must have a 'read' method (file-like object) that is called to fetch the bytes in memory.- If `'content'`, the input is expected to be a sequence of items that can be of type string or byte.",'content'
,"encoding encoding: str, default='utf-8'If bytes or files are given to analyze, this encoding is used todecode.",'utf-8'
,"decode_error decode_error: {'strict', 'ignore', 'replace'}, default='strict'Instruction on what to do if a byte sequence is given to analyze thatcontains characters not of the given `encoding`. By default, it is'strict', meaning that a UnicodeDecodeError will be raised. Othervalues are 'ignore' and 'replace'.",'strict'
,"strip_accents strip_accents: {'ascii', 'unicode'} or callable, default=NoneRemove accents and perform other character normalizationduring the preprocessing step.'ascii' is a fast method that only works on characters that havea direct ASCII mapping.'unicode' is a slightly slower method that works on any characters.None (default) means no character normalization is performed.Both 'ascii' and 'unicode' use NFKD normalization from:func:`unicodedata.normalize`.",None
,"lowercase lowercase: bool, default=TrueConvert all characters to lowercase before tokenizing.",True
,"preprocessor preprocessor: callable, default=NoneOverride the preprocessing (string transformation) stage whilepreserving the tokenizing and n-grams generation steps.Only applies if ``analyzer`` is not callable.",None
,"tokenizer tokenizer: callable, default=NoneOverride the string tokenization step while preserving thepreprocessing and n-grams generation steps.Only applies if ``analyzer == 'word'``.",None


In [26]:
test_preds = best_model.predict(X_test)
print(classification_report(y_test, test_preds))
print(confusion_matrix(y_test, test_preds))


              precision    recall  f1-score   support

           0       1.00      0.98      0.99      2143
           1       0.98      1.00      0.99      2067

    accuracy                           0.99      4210
   macro avg       0.99      0.99      0.99      4210
weighted avg       0.99      0.99      0.99      4210

[[2099   44]
 [   7 2060]]


In [27]:
os.makedirs("../app/model", exist_ok=True)

MODEL_PATH = "../app/model/spam_classifier.joblib"
joblib.dump(best_model, MODEL_PATH)

print("Model saved to:", MODEL_PATH)


Model saved to: ../app/model/spam_classifier.joblib


In [28]:
label_mapping = {0: "ham", 1: "spam"}
joblib.dump(label_mapping, "../app/model/label_mapping.joblib")


['../app/model/label_mapping.joblib']